# Importing the libraries

In [10]:
import pandas as pd
import numpy as np
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
import re
import csv

# Collaborative Filtering (first attempts)
Recommendations based on user-item interactions. 


### Train Test Split

In [11]:
# Load the data
interactions = pd.read_csv('kaggle_data/interactions_train.csv')
items = pd.read_csv('kaggle_data/items.csv')

interactions = interactions.sort_values(["u", "t"])

interactions["pct_rank"] = interactions.groupby("u")["t"].rank(pct=True, method='dense')
interactions.reset_index(inplace=True, drop=True)

train_data = interactions[interactions["pct_rank"] < 0.9]
test_data = interactions[interactions["pct_rank"] >= 0.9]


print("Training set size:", train_data.shape[0])
print("Testing set size:", test_data.shape[0])

Training set size: 73892
Testing set size: 13155


### Collaborative Filtering (simple approach)
Using the initial raw data

In [15]:
# Create a user-item interaction matrix with binary values (1 if read, 0 otherwise)
binary_interaction_matrix = train_data.pivot_table(index='u', columns='i', values='t', aggfunc='count')
binary_interaction_matrix = binary_interaction_matrix.notnull().astype(int)

binary_interaction_matrix

i,0,1,2,3,4,5,6,7,8,9,...,15279,15280,15282,15283,15284,15285,15287,15288,15289,15290
u,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7833,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7834,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7835,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Compute Cosine Similarity Between Users
user_similarity = cosine_similarity(binary_interaction_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=binary_interaction_matrix.index, columns=binary_interaction_matrix.index)
user_similarity_df

u,0,1,2,3,4,5,6,7,8,9,...,7828,7829,7830,7831,7832,7833,7834,7835,7836,7837
u,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.101015
3,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7833,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000
7834,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
7835,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000


In [ ]:
k = 10

# User-item scores for every user in one shot
scores = user_similarity_df.values @ binary_interaction_matrix.values        # (U, I)

# Indices of each user’s k best-scoring items (unsorted)
top_idx_unsorted = np.argpartition(-scores, k-1, axis=1)[:, :k]              # (U, k)

# Sort those k items per user so they’re really rank-ordered
rows   = np.arange(scores.shape[0])[:, None]                                  # (U, 1)
order  = np.argsort(-scores[rows, top_idx_unsorted], axis=1)
top_idx = top_idx_unsorted[rows, order]                                       # (U, k) sorted

# Look up the *labels* with NumPy → 2-D array → DataFrame
item_labels = binary_interaction_matrix.columns.to_numpy()                    # (I,)
top_labels  = item_labels[top_idx]                                            # (U, k)

recommendations = pd.DataFrame(
    top_labels,                      
    index=binary_interaction_matrix.index, 
    columns=range(k)                        
)

# quick peek
print(recommendations.head())

     0    1    2    3    4    5    6    7    8    9
u                                                  
0   13    4   12   15    8   11   14   10    5   17
1   38   34   31   30   37   29   32   33   36   35
2   46   58   53   49   56   64   91   82   71   45
3  149  163  128  143  133  138   40  155  142  156
4  202  203  198  191  193  201  195  197  205  196


In [18]:
# Convert each row of recommendations to a space-separated string
recommendations_str = recommendations.apply(lambda row: ' '.join(row.astype(str)), axis=1)

# Export to CSV with a single-column header
recommendations_str.to_csv('recommendations.csv', index=True, header=['recommendation'])

### Collaborative Filtering (better predictions)

In [19]:
# Apply TruncatedSVD to reduce dimensionality
svd = TruncatedSVD(n_components=50, random_state=42)
user_factors = svd.fit_transform(binary_interaction_matrix.values)
item_factors = svd.components_

# Reconstruct approximate interaction scores
approx_scores = user_factors @ item_factors

# Convert reconstructed scores into a DataFrame (same indexes and columns as original)
approx_scores_df = pd.DataFrame(approx_scores, index=binary_interaction_matrix.index, columns=binary_interaction_matrix.columns)

# Get top 10 items for each user
k = 10
recommendations_svd = approx_scores_df.apply(lambda row: row.nlargest(k).index.tolist(), axis=1)
recommendations_svd.head()

u
0    [611, 46, 4, 8999, 794, 3407, 3811, 685, 13, 2...
1    [611, 789, 4220, 5140, 2959, 769, 796, 176, 33...
2    [46, 323, 56, 2130, 5748, 3055, 66, 8999, 70, 64]
3    [163, 149, 618, 611, 128, 466, 119, 4, 2614, 143]
4    [424, 323, 201, 2225, 428, 423, 976, 422, 324,...
dtype: object

# Collaborative Filtering (the best scores)
Recommendations based on user-item interactions. 

In [ ]:
# Enable or disable the upscaling of individual book weights for users with low book counts
upscale_low_book = True

# Define the threshold for the number of books below which upscaling will be applied
threshold_low_book = 2

In [ ]:
# Load the data
interactions = pd.read_csv('kaggle_data/interactions_train.csv')
items = pd.read_csv('kaggle_data/items.csv')

# Aggregate interactions to calculate the number of unique items each user has interacted with
interactions_agg = interactions.groupby('u').agg(
    {
        'i': pd.Series.nunique,
    }
).reset_index().rename(
    columns={
        'i': 'n_items'
    }
)

# Check the loaded data
# Ensure the interactions dataset contains the required columns and is not empty
assert 'u' in interactions.columns and 'i' in interactions.columns and 't' in interactions.columns, "Interactions file missing required columns"
assert not interactions.empty, "Interactions dataset is empty"

# Ensure the items dataset contains the required columns and is not empty
assert 'Title' in items.columns and 'i' in items.columns, "Items file missing required columns"
assert not items.empty, "Items dataset is empty"

# Following adjustments
interactions = interactions.sort_values(["u", "t"], ascending=[True, False])

# Calculate the percentile rank of each interaction within a user's interactions
interactions["pct_rank"] = interactions.groupby("u")["t"].rank(pct=True, method='dense')
interactions.reset_index(inplace=True, drop=True)

# Merge the interactions dataset with the aggregated data to include the number of unique items per user
interactions = interactions.merge(interactions_agg, on='u', how='left')
interactions['weight'] = interactions["pct_rank"]

# If the upscale_low_book flag is True, assign a weight of 1 to interactions for users with a number of unique items less than or equal to the threshold
if upscale_low_book:
    interactions.loc[interactions['n_items'] <= threshold_low_book, 'weight'] = 1

inter_data = interactions.copy()

# Create binary interaction matrix
binary_interaction_matrix = inter_data.pivot_table(index='u', columns='i', values='weight', aggfunc='sum')
binary_interaction_matrix = binary_interaction_matrix.fillna(0)  # Fill NaN with 0

# Compute cosine similarity and test symmetry
user_similarity = cosine_similarity(binary_interaction_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=binary_interaction_matrix.index, columns=binary_interaction_matrix.index)

assert np.allclose(user_similarity_df, user_similarity_df.T), "User similarity matrix is not symmetric"

# Top-10 Recommendations from this approach
k = 10
scores = user_similarity_df.values @ binary_interaction_matrix.values
top_idx_unsorted = np.argpartition(-scores, k-1, axis=1)[:, :k]              
rows = np.arange(scores.shape[0])[:, None]
order = np.argsort(-scores[rows, top_idx_unsorted], axis=1)
top_idx = top_idx_unsorted[rows, order]
item_labels = binary_interaction_matrix.columns.to_numpy()                    
top_labels = item_labels[top_idx]                                            
recommendations = pd.DataFrame(top_labels, index=binary_interaction_matrix.index, columns=range(k))
assert recommendations.shape[0] == binary_interaction_matrix.shape[0], "Number of recommendation rows differs from number of users"
assert recommendations.shape[1] == k, "Each user should have 10 recommendations"

In [ ]:
# Convert each row of recommendations to a space-separated string
recommendations_str = recommendations.apply(lambda row: ' '.join(row.astype(str)), axis=1)

# Export recommendations to a CSV file
if upscale_low_book:
    recommendations_str.to_csv(
        f'recommendations_collab_weight_pct_upscale_1_nitems_{threshold_low_book}.csv',
        index=True,
        header=['recommendation'],
        index_label='user_id'
    )
else:
    recommendations_str.to_csv(
        'recommendations_collab_weight_pct.csv',
        index=True,
        header=['recommendation'],
        index_label='user_id'
    )

# recommendations_collab_weight_pct_upscale_1_nitems_2.csv = 0.1642 (the best score)
# recommendations_collab_weight_pct.csv = 0.1637 (the second best score): without upscaling

# TFIDF-Based Book Recommendations (not cleaned dataset)
This section demonstrates how to generate book recommendations for each user based on the similarity of books they have already read using the TFIDF approach

In [ ]:
# Create a TF-IDF vectorizer using French stop words and fit-transform the 'combined_features' column of the books
tfidf_vectorizer = TfidfVectorizer(stop_words=french_stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(books['combined_features'])

In [ ]:
# naive_limit_bool = True, if you want to limit the recommendations, otherwise set it to False
naive_limit_bool = True

interactions_agg = interactions.groupby('u').agg(
    min_book_id = ('i', min),
    max_book_id = ('i', max),
).reset_index()


# Compute recommendations for each user
user_recommendations = {}
for user_id, group in interactions.groupby('u'):
    read_books = group['i'].values
    read_books_indices = [books[books['i'] == book_id].index[0] for book_id in read_books]
    read_books_tfidf = tfidf_matrix[read_books_indices]
    similarity_scores = cosine_similarity(read_books_tfidf, tfidf_matrix)
    avg_similarity = np.mean(similarity_scores, axis=0)
    recommended_indices = avg_similarity.argsort()[-15288:][::-1]

    recommended_books = books.iloc[recommended_indices]['i'].values

    if naive_limit_bool:
        lower_bound = interactions_agg[interactions_agg['u'] == user_id]['min_book_id'].values[0]
        upper_bound = interactions_agg[interactions_agg['u'] == user_id]['max_book_id'].values[0]
        
        if upper_bound > lower_bound:
            upper_bound = min(upper_bound+11, 15290)
            recommended_books = recommended_books[(recommended_books >= lower_bound) & (recommended_books <= upper_bound)]

        if len(recommended_books) < 10:
            print(f"User {user_id} has less than 10 recommendations. Found: {len(recommended_books)}")
            print("upper_bound:", upper_bound)
            print("lower_bound:", lower_bound)
    
    user_recommendations[user_id] = recommended_books[:10]

In [ ]:
# Display recommendations for a sample user
sample_user_id = list(user_recommendations.keys())[0]
print(f"Recommendations for User {sample_user_id}:")
print(user_recommendations[sample_user_id])

In [ ]:
# Export recommendations to a CSV file
with open('recommendations_tfidf_naive_proper_input_less_features.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['user_id', 'recommendation'])
    for user_id, recommended_books in user_recommendations.items():
        writer.writerow([user_id, " ".join(map(str, recommended_books))])

# Score = 0.1560
print("Recommendations exported to recommendations_tfidf_naive_proper_input_less_features.csv")

In [ ]:
# For each user, who has read only one unique book (does not matter once or several times), recommend 10 times the same book
user_embeddings_df = interactions.copy()

user_embeddings_df = user_embeddings_df.sort_values(by=["u", "t"],ascending=[True, False]).reset_index(drop=True)
user_embeddings_df['last_unique_books'] = user_embeddings_df\
        .groupby('u')['i']\
        .apply(lambda x: (~pd.Series(x).duplicated()).cumsum()).reset_index(drop=True)

# Get users who have read exactly 1 unique book
users_with_1_unique_book = user_embeddings_df.groupby('u')['i'].nunique()
users_with_1_unique_book = users_with_1_unique_book[users_with_1_unique_book == 1].index
u_1 = user_embeddings_df[user_embeddings_df['u'].isin(users_with_1_unique_book)].drop_duplicates(subset=['u', 'i'])
u_1

In [ ]:
for user_id in u_1['u'].unique():
    book_id = u_1[u_1['u'] == user_id]['i'].values[0]
    # Create a recommendation list with 10 copies of the book ID
    user_recommendations[user_id] = [book_id] * 10

In [ ]:
# Export recommendations to a CSV file
with open('recommendations_tfidf_naive_proper_input_less_features_1.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['user_id', 'recommendation'])
    for user_id, recommended_books in user_recommendations.items():
        writer.writerow([user_id, " ".join(map(str, recommended_books))])

# Score = 0.1558
print("Recommendations exported to recommendations_tfidf_naive_proper_input_less_features_1.csv")